<a href="https://colab.research.google.com/github/estebangaviria/Data-Science/blob/main/Regresi%C3%B3n_Logistica_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modelo de clasificación por regresión logística- Cancer

El script debe realizar las siguientes actividades:

1. Cargar el conjunto de datos en un data frame de pandas.
2. Separar los datos en conjunto de entrenamiento y prueba.
3. Construir un modelo a partir de los datos de entrenamiento (construir un modelo de clasificación por regresión logística).
4. Probar el modelo utilizando los datos de prueba. Reportar la medida accuracy para el modelo construido.

### 1. Carga de datos 

In [ ]:
# Carga de librerías y lectura del archivo que contiene los datos

import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

url="https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
data = pd.read_csv(url, header=None, na_values=' ?')

# Se etiquetan las columnas para poder interpretar más fácilmente los datos.
# Cada fila es un ejemplo, cada columna un atributo.
# La columna llamada C es el atributo a predecir
data.columns = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei',
              'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']
data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [ ]:
data.pop('Sample code number')
data.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


#### Análisis de datos

In [ ]:
#Cuál es el número de registros?
#Cuál es el número de atributos?

shape = data.shape
shape

(699, 10)

In [ ]:
#Cual es el tipo de los atributos?
data.dtypes

Clump Thickness                int64
Uniformity of Cell Size        int64
Uniformity of Cell Shape       int64
Marginal Adhesion              int64
Single Epithelial Cell Size    int64
Bare Nuclei                    int64
Bland Chromatin                int64
Normal Nucleoli                int64
Mitoses                        int64
Class                          int64
dtype: object

In [ ]:
#Medida de centralidad y desviación para atributos numéricos:
data.describe()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.486409,3.437768,2.866953,1.589413,0.344778
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.621929,2.438364,3.053634,1.715078,0.475636
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,0.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [ ]:
#Medida de centralidad para atributos categóricos:
data.mode()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1,1,1,1,2,1,2,1,1,0


In [ ]:
pd.unique(data['Bare Nuclei'])

array([ 1, 10,  2,  4,  3,  9,  7,  5,  8,  6])

In [ ]:
# Se reemplazan los datos erroneos por la moda del atributo Bare Nuclei
data = data.replace({'?': 1})

In [ ]:
data['Bare Nuclei'] = data['Bare Nuclei'].astype('int64')

In [ ]:
data.dtypes

Clump Thickness                int64
Uniformity of Cell Size        int64
Uniformity of Cell Shape       int64
Marginal Adhesion              int64
Single Epithelial Cell Size    int64
Bare Nuclei                    int64
Bland Chromatin                int64
Normal Nucleoli                int64
Mitoses                        int64
Class                          int64
dtype: object

In [ ]:
#Medida de centralidad para atributos categóricos:
data.mode()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1,1,1,1,2,1,2,1,1,0


In [ ]:
from sklearn.preprocessing import LabelEncoder 

# Se convierten los atributos categóricos a valores numéricos
labelencoder= LabelEncoder() 
data['Class'] = labelencoder.fit_transform(data['Class'])
data.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [ ]:
#Cuantos registros hay por cada clase? es decir, por cada valor del atributo de salida?
print(data['Class'].value_counts())

0    458
1    241
Name: Class, dtype: int64


In [ ]:
# Se hace balanceo de clases eliminando ejemplos de la clase mayoritaria
g = data.groupby('Class')
dataBal = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))
dataBal

Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
Class                                                                           
0     0                  6                        8                         8   
      1                  1                        1                         1   
      2                  1                        1                         1   
      3                  3                        1                         1   
      4                  5                        1                         1   
...                    ...                      ...                       ...   
1     236               10                        8                        10   
      237                8                       10                        10   
      238                8                        6                         4   
      239                9                       10                        10   
      240               10                        4                         3   

           Marginal Adhesion  Single Epithelial Cell Size  Bare Nuclei  \
Class                                                                    
0     0                    1                            3            4   
      1                    1                            2            1   
      2                    1                            2            1   
      3                    1                            2            1   
      4                    1                            2            1   
...                      ...                          ...          ...   
1     236                 10                            6            1   
      237                 10                            8           10   
      238                  3                            5            9   
      239                 10                           10            5   
      240                 10                            4           10   

           Bland Chromatin  Normal Nucleoli  Mitoses  Class  
Class                                                        
0     0                  3                7        1      0  
      1                  1                1        1      0  
      2                  1                1        1      0  
      3                  3                1        1      0  
      4                  2                1        1      0  
...                    ...              ...      ...    ...  
1     236                3                1       10      1  
      237               10                7        3      1  
      238                3                1        1      1  
      239               10               10       10      1  
      240               10                1        1      1  

[482 rows x 10 columns]

In [ ]:
dataBal.describe()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000
mean,5.070539,3.941909,4.006224,3.435685,3.705394,4.410788,4.018672,3.568465,1.829876,0.500000
std,2.971662,3.333311,3.222511,3.165073,2.421390,3.955749,2.637027,3.370485,1.997946,0.500519
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000
50%,5.000000,3.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,1.000000,0.500000
75%,8.000000,6.750000,7.000000,5.000000,5.000000,10.000000,7.000000,6.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [ ]:
# Se verifica que haya quedado el mismo número de registros por cada clase
print(dataBal['Class'].value_counts())

0    241
1    241
Name: Class, dtype: int64


In [ ]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataBal.drop('Class', axis=1), dataBal.Class, test_size=0.30)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(337, 9) (337,)
(145, 9) (145,)


In [ ]:
# Entrenamiento del modelo de clasificación por regresión logística
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logisticRegr = LogisticRegression(solver="lbfgs", max_iter=500, tol=0.01)
logisticRegr.fit(X_train, y_train)

LogisticRegression(max_iter=500, tol=0.01)

In [ ]:
# Aplicación del modelo construido a los datos de test
predictions = logisticRegr.predict(X_test)
predictions

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0])

In [ ]:
# Cálculo del accuracy para evaluar el desempeño del modelo sobre los datos de test
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)

0.9793103448275862